# Swing Trading Analysis - Getting Started

This notebook demonstrates basic data collection and analysis for swing trading across India, Malaysia, and USA markets.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Collection Setup

First, let's set up our data collection system to gather market data from all three regions.

In [ ]:
# Import our custom modules
import sys
import os
sys.path.append('../')

from tools.data_collector import MarketDataCollector
from tools.technical_analysis import TechnicalAnalyzer, SwingTradingSignals
from strategies.sma_crossover import SMACrossoverStrategy

# Initialize our data collector
collector = MarketDataCollector()
print("Data collector initialized!")

## 2. Sample Data Analysis

Let's analyze a sample stock to demonstrate our technical analysis capabilities.

In [ ]:
# For demonstration, let's create sample data
# In practice, you would load real market data

# Generate sample stock data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end='2024-08-29', freq='D')
n_days = len(dates)

# Simulate stock price movements
price_changes = np.random.normal(0.001, 0.02, n_days)  # Daily returns
prices = [100]  # Starting price

for change in price_changes[1:]:
    new_price = prices[-1] * (1 + change)
    prices.append(new_price)

# Create sample OHLC data
sample_data = pd.DataFrame({
    'Date': dates,
    'Open': prices,
    'High': [p * (1 + abs(np.random.normal(0, 0.01))) for p in prices],
    'Low': [p * (1 - abs(np.random.normal(0, 0.01))) for p in prices],
    'Close': prices,
    'Volume': np.random.randint(1000000, 5000000, n_days)
})

sample_data.set_index('Date', inplace=True)
print(f"Sample data created with {len(sample_data)} days")
sample_data.head()

## 3. Technical Analysis

Now let's apply our technical analysis tools to generate trading signals.

In [ ]:
# Initialize technical analyzer
analyzer = TechnicalAnalyzer()

# Calculate technical indicators
sample_data['SMA_20'] = analyzer.sma(sample_data['Close'], 20)
sample_data['SMA_50'] = analyzer.sma(sample_data['Close'], 50)
sample_data['RSI'] = analyzer.rsi(sample_data['Close'])

# MACD
macd_data = analyzer.macd(sample_data['Close'])
sample_data['MACD'] = macd_data['macd']
sample_data['MACD_Signal'] = macd_data['signal']

print("Technical indicators calculated!")
sample_data[['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD']].tail()

## 4. Strategy Backtesting

Let's test our SMA Crossover strategy on the sample data.

In [ ]:
# Initialize and run SMA Crossover strategy
strategy = SMACrossoverStrategy(fast_period=20, slow_period=50)
results = strategy.backtest(sample_data)

# Display results
print("=" * 50)
print("BACKTESTING RESULTS")
print("=" * 50)
print(f"Strategy: {results['strategy_name']}")
print(f"Total Trades: {results['total_trades']}")
print(f"Buy Signals: {results['buy_signals']}")
print(f"Sell Signals: {results['sell_signals']}")
print(f"Buy & Hold Return: {results['buy_and_hold_return']:.2%}")
print(f"Strategy Return: {results['strategy_return']:.2%}")
print(f"Sharpe Ratio: {results['sharpe_ratio']:.3f}")
print(f"Max Drawdown: {results['max_drawdown']:.2%}")

## 5. Visualization

Let's create some charts to visualize our analysis.

In [ ]:
# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Price and Moving Averages
ax1.plot(sample_data.index, sample_data['Close'], label='Close Price', linewidth=1)
ax1.plot(sample_data.index, sample_data['SMA_20'], label='SMA 20', alpha=0.7)
ax1.plot(sample_data.index, sample_data['SMA_50'], label='SMA 50', alpha=0.7)
ax1.set_title('Price and Moving Averages')
ax1.legend()
ax1.grid(True, alpha=0.3)

# RSI
ax2.plot(sample_data.index, sample_data['RSI'], color='purple')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
ax2.set_title('RSI (Relative Strength Index)')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

# MACD
ax3.plot(sample_data.index, sample_data['MACD'], label='MACD', color='blue')
ax3.plot(sample_data.index, sample_data['MACD_Signal'], label='Signal', color='red')
ax3.bar(sample_data.index, macd_data['histogram'], label='Histogram', alpha=0.3)
ax3.set_title('MACD')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Strategy Returns
strategy_data = results['data']
ax4.plot(strategy_data.index, strategy_data['Cumulative_Return'], 
         label='Buy & Hold', linewidth=2)
ax4.plot(strategy_data.index, strategy_data['Strategy_Cumulative_Return'], 
         label='SMA Strategy', linewidth=2)
ax4.set_title('Strategy Performance Comparison')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Next Steps

This notebook provides a foundation for swing trading analysis. Here are suggested next steps:

1. **Real Data Collection**: Replace sample data with real market data using the data collector
2. **More Strategies**: Implement additional trading strategies (RSI divergence, Bollinger Bands, etc.)
3. **Multi-Asset Analysis**: Analyze multiple stocks across different countries
4. **Risk Management**: Add position sizing and risk management rules
5. **Machine Learning**: Integrate ML models for pattern recognition and prediction
6. **Real-time Analysis**: Set up real-time data feeds and automated signal generation

Remember to:
- Always backtest strategies before using real money
- Consider transaction costs and slippage
- Implement proper risk management
- Stay updated with market conditions and news